In [3]:

import warnings
#from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import pickle
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
def get_stock_data(tickerI):
    tickerL = tickerI
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_is_df = pd.DataFrame(response.json())
    if annual_is_df.shape == (0, 0):
        annual_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_annual = float('Nan')
    else:
        last_date_annual = annual_is_df.loc[0,'date']
        annual_is_df = annual_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_is_df = annual_is_df.add_suffix(f'(FY)')
        
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_is_df = pd.DataFrame(response.json())
    if quarterly_is_df.shape == (0, 0):
        quarterly_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_quarterly = float('Nan')
    else:
        last_date_quarterly = quarterly_is_df.loc[0,'date']
        quarterly_is_df = quarterly_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_is_df = quarterly_is_df.add_suffix(f'(FQ)')
        
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_fp_df = pd.DataFrame(response.json())
    if annual_fp_df.shape == (0, 0):
        annual_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_fp_df = annual_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_fp_df = annual_fp_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_fp_df = pd.DataFrame(response.json())
    if quarterly_fp_df.shape == (0, 0):
        quarterly_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_fp_df = quarterly_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_fp_df = quarterly_fp_df.add_suffix(f'(FQ)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_cf_df = pd.DataFrame(response.json())
    if annual_cf_df.shape == (0, 0):
        annual_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_cf_df = annual_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_cf_df = annual_cf_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_cf_df = pd.DataFrame(response.json())
    if quarterly_cf_df.shape == (0, 0):
        quarterly_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_cf_df = quarterly_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_cf_df = quarterly_cf_df.add_suffix(f'(FQ)')
    
    stock_data = pd.concat([annual_is_df, quarterly_is_df, annual_fp_df, quarterly_fp_df, annual_cf_df  ], axis = 1)
    stock_data = pd.concat([stock_data,quarterly_cf_df], axis = 1)
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='calendarYear')))]
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='period')))]
    stock_data = stock_data.loc[:,~stock_data.columns.duplicated()].copy()
    stock_data1 = stock_data.iloc[0:9,]
    
    if quarterly_cf_df.shape[0]<9:
        for year in range(1,quarterly_cf_df.shape[0]):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    else:
        for year in range(1,9):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    stock_data1 = stock_data1.iloc[0,:]
    stock_data_final = pd.DataFrame(stock_data1).transpose()
    stock_data_final.insert(loc = 0, column ="Ticker", value=f"{tickerL}" )
    stock_data_final.insert(loc = 0, column ="Date(FY)", value=f"{last_date_annual}" )
    stock_data_final.insert(loc = 0, column ="Date(FQ)", value=f"{last_date_quarterly}" )
    
    return stock_data_final

In [3]:
us_stock_database = pd.DataFrame()
stock_list_pd = pd.read_pickle("StockList")
stock_list_pd = stock_list_pd.query("type == 'stock'")
stock_list_pd = stock_list_pd.query("exchangeShortName == 'NYSE' | exchangeShortName == 'NASDAQ'")
stock_list_pd.reset_index(inplace=True, drop=True)
stock_list = stock_list_pd["symbol"].to_list()
for ticker in tqdm(stock_list):
    #tqdm._instances.clear()
    us_stock_database = pd.concat([us_stock_database, get_stock_data(ticker)])
    #print(f'processed {ticker}')
    us_stock_database.reset_index(drop=True)


100%|██████████| 10126/10126 [21:10:38<00:00,  7.53s/it]  


In [4]:
db = pd.read_parquet('USDBPARQ.parquet')

In [5]:
db

,revenue(FY),costOfRevenue(FY),grossProfit(FY),grossProfitRatio(FY),researchAndDevelopmentExpenses(FY),generalAndAdministrativeExpenses(FY),sellingAndMarketingExpenses(FY),sellingGeneralAndAdministrativeExpenses(FY),otherExpenses(FY),operatingExpenses(FY),...,operatingIncomeRatio(FY)-1-8,totalOtherIncomeExpensesNet(FY)-1-8,incomeBeforeTax(FY)-1-8,incomeBeforeTaxRatio(FY)-1-8,incomeTaxExpense(FY)-1-8,netIncome(FY)-1-8,netIncomeRatio(FY)-1-8,eps(FY)-1-8,Date(FQ),Date(FY)
Ticker,,,,,,,,,,,,,,,,,,,,,
NGL-PB,8.694904e+09,7.923645e+09,7.712590e+08,0.088702,0.0,7.181800e+07,0.000000e+00,7.181800e+07,3.233900e+08,3.952080e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31,2023-03-31
NLY-PI,-1.651591e+09,2.480600e+07,-1.676397e+09,1.015019,0.0,4.296100e+07,0.000000e+00,4.296100e+07,-3.339364e+09,4.296100e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31,2023-12-31
CIO-PA,1.790960e+08,6.999700e+07,1.090990e+08,0.609165,0.0,1.484100e+07,0.000000e+00,1.484100e+07,5.515600e+07,7.782800e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31,2023-12-31
LEVI,6.179000e+09,2.663300e+09,3.515700e+09,0.568976,0.0,1.267600e+09,1.803300e+09,3.072200e+09,-3.500000e+06,3.072200e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-25,2023-11-26
ICL,7.654434e+09,5.074516e+09,2.579918e+09,0.337049,72115816.0,4.360000e+08,9.170000e+08,1.286912e+09,-4.000000e+06,1.359028e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31,2023-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UNVR,1.147530e+10,8.704100e+09,2.771200e+09,0.241493,0.0,1.282800e+09,0.000000e+00,1.282800e+09,6.563000e+08,1.939100e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-30,2022-12-31
GOGN-WT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,5.397402e+06,0.000000e+00,5.397402e+06,0.000000e+00,5.397402e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-31,2022-12-31
VHNAW,1.356050e+07,5.413686e+06,8.146812e+06,0.600775,2670333.0,4.648242e+06,1.073617e+07,1.538442e+07,-6.358000e+03,1.967896e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31,2023-03-31


In [8]:
ticker = 'AAPL'
aapl_df = db.query(f"Ticker == '{ticker}'")
aapl_df

,revenue(FY),costOfRevenue(FY),grossProfit(FY),grossProfitRatio(FY),researchAndDevelopmentExpenses(FY),generalAndAdministrativeExpenses(FY),sellingAndMarketingExpenses(FY),sellingGeneralAndAdministrativeExpenses(FY),otherExpenses(FY),operatingExpenses(FY),...,operatingIncomeRatio(FY)-1-8,totalOtherIncomeExpensesNet(FY)-1-8,incomeBeforeTax(FY)-1-8,incomeBeforeTaxRatio(FY)-1-8,incomeTaxExpense(FY)-1-8,netIncome(FY)-1-8,netIncomeRatio(FY)-1-8,eps(FY)-1-8,Date(FQ),Date(FY)
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,3.832850e+11,2.141370e+11,1.691480e+11,0.441311,2.991500e+10,0.0,0.0,2.493200e+10,-565000000.0,5.484700e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-30,2023-09-30


In [117]:
period = 'FY'
date_list = []
last_report_date = aapl_df.loc[f'{ticker}',f'Date({period})']
date_list.append(last_report_date)
for year in range(1,5):
    date = pd.to_datetime(last_report_date) - relativedelta(years=year)
    date_list.append(date.date().strftime("%Y-%m-%d"))
date_list

['2023-09-30', '2022-09-30', '2021-09-30', '2020-09-30', '2019-09-30']

In [97]:
ratios = ['Date_Index','ROA', 'ROE', 'ROIC', 'GM%', 'NPM%']

In [98]:
date_index = [0, 1, 2, 3, 4]

In [99]:
ratio_df  = pd.DataFrame(index =ratios, columns=date_list)
ratio_df.loc['Date_Index'] = date_index
#ratio_df.index = ['ROA', 'ROE', 'ROIC', 'GM%', 'OPM%', 'NM%']
ratio_df

,2023-09-30,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Date_Index,0,1,2,3,4
ROA,NaN,NaN,NaN,NaN,NaN
ROE,NaN,NaN,NaN,NaN,NaN
ROIC,NaN,NaN,NaN,NaN,NaN
GM%,NaN,NaN,NaN,NaN,NaN
NPM%,NaN,NaN,NaN,NaN,NaN


In [108]:
historical_ratio_df = ratio_df.iloc[:,1:]
current_ratio_df = pd.DataFrame(ratio_df.iloc[:,0])
for year in historical_ratio_df.columns.to_list():
    for ratio in historical_ratio_df.index.to_list()[1:]:
        year_index = (historical_ratio_df[year]['Date_Index'])
        if ratio == "ROA":
            historical_ratio_df[year][ratio] = (aapl_df[f'revenue({period})-{year_index}'][0]/aapl_df[f'totalAssets({period})-{year_index}'][0])*100
            
for year in current_ratio_df.columns.to_list():
    for ratio in current_ratio_df.index.to_list()[1:]:
        current_ratio_df[last_report_date][ratio] = (aapl_df[f'revenue({period})'][0]/aapl_df[f'totalAssets({period})'][0])*100

/var/folders/y7/q3m4z5nn3xz3qjh1fqwvrpvh0000gn/T/ipykernel_76152/746419292.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_ratio_df[year][ratio] = (aapl_df[f'revenue({period})-{year_index}'][0]/aapl_df[f'totalAssets({period})-{year_index}'][0])*100


In [107]:
current_ratio_df['2023-09-30']['ROA']

nan

In [103]:
historical_ratio_df

,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Date_Index,1,2,3,4
ROA,111.785233,104.220774,84.75615,76.857224
ROE,111.785233,104.220774,84.75615,76.857224
ROIC,111.785233,104.220774,84.75615,76.857224
GM%,111.785233,104.220774,84.75615,76.857224
NPM%,111.785233,104.220774,84.75615,76.857224


In [110]:
current_ratio_df

,2023-09-30
Date_Index,0
ROA,108.707737
ROE,108.707737
ROIC,108.707737
GM%,108.707737
NPM%,108.707737


In [114]:
ratio_df = pd.concat([current_ratio_df,historical_ratio_df], axis=1)

In [115]:
ratio_df

,2023-09-30,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Date_Index,0,1,2,3,4
ROA,108.707737,111.785233,104.220774,84.75615,76.857224
ROE,108.707737,111.785233,104.220774,84.75615,76.857224
ROIC,108.707737,111.785233,104.220774,84.75615,76.857224
GM%,108.707737,111.785233,104.220774,84.75615,76.857224
NPM%,108.707737,111.785233,104.220774,84.75615,76.857224
